# Importing the packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt

from sklearn import set_config
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error

sns.set_style("whitegrid")
set_config(transform_output='pandas')

# Loading the data

In [2]:
df_jan_path = "../data/raw/yellow_tripdata_2016-01.csv"
df_feb_path = "../data/raw/yellow_tripdata_2016-02.csv"
df_mar_path = "../data/raw/yellow_tripdata_2016-03.csv"

usecols = [
    "trip_distance", "tpep_pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "fare_amount"
]

df_jan = dd.read_csv(df_jan_path, assume_missing=True, usecols=usecols, parse_dates=["tpep_pickup_datetime"])
df_feb = dd.read_csv(df_feb_path, assume_missing=True, usecols=usecols, parse_dates=["tpep_pickup_datetime"])
df_mar = dd.read_csv(df_mar_path, assume_missing=True, usecols=usecols, parse_dates=["tpep_pickup_datetime"])

df_jan

,tpep_pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount
npartitions=26,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


## Concatenating the data

In [3]:
df = dd.concat([df_jan, df_feb, df_mar], axis=0)
df

,tpep_pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount
npartitions=82,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


# Bounds

In [4]:
min_lat = 40.60
max_lat = 40.85
min_lon = -74.05
max_lon = -73.70

min_fare_amount = 0.50
max_fare_amount = 81.0

min_trip_distance = 0.25
max_trip_distance = 24.43

# Removing outliers and unnecessary columns

In [5]:
df = df.loc[
    (df["pickup_latitude"].between(min_lat, max_lat, inclusive="both")) & 
    (df["pickup_longitude"].between(min_lon, max_lon, inclusive="both")) & 
    (df["dropoff_latitude"].between(min_lat, max_lat, inclusive="both")) & 
    (df["dropoff_longitude"].between(min_lon, max_lon, inclusive="both")),
    :
]
df = df.loc[
    (df["fare_amount"].between(min_fare_amount, max_fare_amount, inclusive="both")) & 
    (df["trip_distance"].between(min_trip_distance, max_trip_distance, inclusive="both")),
    :
]

df = df.drop(columns=["trip_distance", "dropoff_longitude", "dropoff_latitude", "fare_amount"])
df

,tpep_pickup_datetime,pickup_longitude,pickup_latitude
npartitions=82,,,
,datetime64[ns],float64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


# Saving the dataframe

In [6]:
df = df.compute()

In [7]:
INTERIM_DATA_PATH = "../data/interim/processing_data.csv"
# df.to_csv(INTERIM_DATA_PATH, index=False)

# Reading the saved dataframe and clustering

In [8]:
df_reader = pd.read_csv(INTERIM_DATA_PATH, chunksize=100000, usecols=["pickup_latitude", "pickup_longitude"])

In [9]:
scaler = StandardScaler()

for chunk in df_reader:
    scaler.partial_fit(chunk)

In [10]:
scaler

StandardScaler()

In [11]:
df_reader = pd.read_csv(INTERIM_DATA_PATH, chunksize=100000, usecols=["pickup_latitude", "pickup_longitude"])

In [12]:
mini_batch_k_means = MiniBatchKMeans(n_clusters=30, n_init=20, random_state=42)

for chunk in df_reader:
    scaled_chunk = scaler.transform(chunk)
    mini_batch_k_means.partial_fit(scaled_chunk)

In [13]:
mini_batch_k_means

MiniBatchKMeans(n_clusters=30, n_init=20, random_state=42)

In [14]:
scaler.inverse_transform(mini_batch_k_means.cluster_centers_)

array([[-73.94975046,  40.80392392],
       [-73.97685385,  40.74726528],
       [-73.92094427,  40.69670159],
       [-73.78474354,  40.64663525],
       [-74.00391496,  40.7193993 ],
       [-73.97901919,  40.7625422 ],
       [-73.8689102 ,  40.77072843],
       [-73.9884009 ,  40.73666362],
       [-73.95328925,  40.78020052],
       [-73.98620137,  40.68996945],
       [-73.98889533,  40.72380321],
       [-73.9778017 ,  40.78270006],
       [-73.99461378,  40.74864784],
       [-73.91560827,  40.75973368],
       [-73.98338682,  40.74983178],
       [-73.98706818,  40.75674608],
       [-73.95483503,  40.71519695],
       [-73.94394974,  40.82705049],
       [-73.96930766,  40.75888759],
       [-73.98421995,  40.77097094],
       [-74.01111796,  40.70921009],
       [-73.95941789,  40.76949887],
       [-74.00151635,  40.73026981],
       [-73.96899532,  40.79527921],
       [-73.9900069 ,  40.66723527],
       [-73.993915  ,  40.76061753],
       [-73.98061923,  40.73552684],
 

In [15]:
location_subset = df[df.columns[1:]]
location_subset

,pickup_longitude,pickup_latitude
0,-73.990372,40.734695
1,-73.980782,40.729912
2,-73.984550,40.679565
3,-73.993469,40.718990
4,-73.960625,40.781330
...,...,...
420269,-73.790565,40.644451
420270,-73.788055,40.641483
420271,-73.789154,40.646736
420273,-73.977356,40.774471


In [16]:
scaled_location_subset = scaler.transform(location_subset)
scaled_location_subset

,pickup_longitude,pickup_latitude
0,-0.443778,-0.601549
1,-0.182839,-0.777271
2,-0.285388,-2.626699
3,-0.528059,-1.178462
4,0.365612,1.111533
...,...,...
420269,4.992775,-3.916590
420270,5.061072,-4.025611
420271,5.031179,-3.832653
420273,-0.089631,0.859580


In [17]:
cluster_predictions = mini_batch_k_means.predict(scaled_location_subset)
cluster_predictions.shape

(33234199,)

In [18]:
# save the cluster predictions in data

df['region'] = cluster_predictions
df

,tpep_pickup_datetime,pickup_longitude,pickup_latitude,region
0,2016-01-01 00:00:00,-73.990372,40.734695,7
1,2016-01-01 00:00:00,-73.980782,40.729912,26
2,2016-01-01 00:00:00,-73.984550,40.679565,9
3,2016-01-01 00:00:00,-73.993469,40.718990,10
4,2016-01-01 00:00:00,-73.960625,40.781330,8
...,...,...,...,...
420269,2016-03-31 21:43:11,-73.790565,40.644451,3
420270,2016-03-20 08:45:16,-73.788055,40.641483,3
420271,2016-03-20 08:59:21,-73.789154,40.646736,3
420273,2016-03-26 03:02:32,-73.977356,40.774471,19


This has assigned a region to each row in the data according to the clusters.

In [19]:
# Dropping the latitude and longitude columns

time_series_data = df.drop(columns=["pickup_latitude", "pickup_longitude"])
time_series_data

,tpep_pickup_datetime,region
0,2016-01-01 00:00:00,7
1,2016-01-01 00:00:00,26
2,2016-01-01 00:00:00,9
3,2016-01-01 00:00:00,10
4,2016-01-01 00:00:00,8
...,...,...
420269,2016-03-31 21:43:11,3
420270,2016-03-20 08:45:16,3
420271,2016-03-20 08:59:21,3
420273,2016-03-26 03:02:32,19


In [20]:
time_series_data.dtypes

tpep_pickup_datetime    datetime64[ns]
region                           int32
dtype: object

In [21]:
# Saving the time series data

TIME_SERIES_DATA_PATH = "../data/interim/time_series.csv"

# time_series_data.to_csv(TIME_SERIES_DATA_PATH, index=False)

In [22]:
# Setting the time series column as the index

time_series_data.set_index("tpep_pickup_datetime", inplace=True)
time_series_data

,region
tpep_pickup_datetime,
2016-01-01 00:00:00,7
2016-01-01 00:00:00,26
2016-01-01 00:00:00,9
2016-01-01 00:00:00,10
2016-01-01 00:00:00,8
...,...
2016-03-31 21:43:11,3
2016-03-20 08:45:16,3
2016-03-20 08:59:21,3


In [23]:
# Checking for missing values

time_series_data.isna().sum()

region    0
dtype: int64

In [24]:
# Grouping the data by regions

region_grp = time_series_data.groupby("region")
region_grp

In [25]:
# Resampling the time series in 15 min intervals

resampled_time_series_data = (
    region_grp["region"]
    .resample("15min")
    .count()
)
resampled_time_series_data.name = "total_pickups"
resampled_time_series_data = resampled_time_series_data.reset_index(level=0)
resampled_time_series_data

,region,total_pickups
tpep_pickup_datetime,,
2016-01-01 00:00:00,0,58
2016-01-01 00:15:00,0,120
2016-01-01 00:30:00,0,149
2016-01-01 00:45:00,0,160
2016-01-01 01:00:00,0,187
...,...,...
2016-03-31 22:45:00,29,14
2016-03-31 23:00:00,29,17
2016-03-31 23:15:00,29,18


In [26]:
# Checking number of zeros in the data

(resampled_time_series_data["total_pickups"] == 0).sum()

np.int64(3668)

There are 3,668 15-minute time intervals that have 0 pickups. I will replace them with an arbitrary value. I did this because the `mean_absolute_percentage_error` function from sklearn does not perform well with 0 or close to 0 values in `y_true`.

In [27]:
epsilon_val = 10
resampled_time_series_data.replace({'total_pickups': {0 : epsilon_val}}, inplace=True)

(resampled_time_series_data["total_pickups"] == 0).sum()

np.int64(0)

# Smoothing

## Moving average

In [28]:
window_sizes = list(range(3, 11, 1))
window_sizes

[3, 4, 5, 6, 7, 8, 9, 10]

In [29]:
def calculate_best_window_size(window_sizes: list):
    for window in window_sizes:
        idx = window - 1
        y_pred = resampled_time_series_data["total_pickups"].rolling(window=window).mean().values[idx:]
        y_true = resampled_time_series_data["total_pickups"].values[idx:]
        mape = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
        print(f"For the window size {window}, MAPE is {mape*100:.2f}%.")

In [30]:
calculate_best_window_size(window_sizes)

For the window size 3, MAPE is 19.72%.
For the window size 4, MAPE is 23.80%.
For the window size 5, MAPE is 27.55%.
For the window size 6, MAPE is 31.25%.
For the window size 7, MAPE is 34.94%.
For the window size 8, MAPE is 38.63%.
For the window size 9, MAPE is 42.39%.
For the window size 10, MAPE is 46.25%.


One thing that is clear is that to predict number of pickups, I need to give more importance to the more recent values as compared to the values that are more in the past.

## EWMA

In [31]:
resampled_time_series_data["total_pickups"].ewm(alpha=0.9).mean()

tpep_pickup_datetime
2016-01-01 00:00:00     58.000000
2016-01-01 00:15:00    114.363636
2016-01-01 00:30:00    145.567568
2016-01-01 00:45:00    158.558056
2016-01-01 01:00:00    184.156062
                          ...    
2016-03-31 22:45:00     14.720768
2016-03-31 23:00:00     16.772077
2016-03-31 23:15:00     17.877208
2016-03-31 23:30:00     13.487721
2016-03-31 23:45:00     13.948772
Name: total_pickups, Length: 262080, dtype: float64

In [32]:
alpha_values = np.arange(0.2, 1, 0.1)
alpha_values

array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [33]:
def calculate_best_alpha_value(alpha_values):
    y_true = resampled_time_series_data["total_pickups"].values
    for alpha in alpha_values:
        y_pred = resampled_time_series_data["total_pickups"].ewm(alpha=alpha).mean()
        mape = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
        print(f"For alpha = {alpha:.1f}, MAPE is {mape*100:.2f}%.")

In [34]:
calculate_best_alpha_value(alpha_values)

For alpha = 0.2, MAPE is 40.63%.
For alpha = 0.3, MAPE is 27.49%.
For alpha = 0.4, MAPE is 20.41%.
For alpha = 0.5, MAPE is 15.64%.
For alpha = 0.6, MAPE is 11.91%.
For alpha = 0.7, MAPE is 8.70%.
For alpha = 0.8, MAPE is 5.73%.
For alpha = 0.9, MAPE is 2.87%.


Clearly, more the `alpha` value, better the MAPE. This again reinforces that I should give more weight to the recent values. I will choose $\alpha = 0.4$ because this value is supposed to be chosen taking into consideration the number of lag features in the data. I am going to choose 4 lag features. For $n$ lag features, the value of $\alpha$ is given by
$$
\alpha = \frac{2}{n + 1}
$$
Substituting $n = 4$, we get $\alpha = 0.4$.

In [35]:
# Applying smoothing

resampled_time_series_data["avg_pickups"] = resampled_time_series_data["total_pickups"].ewm(alpha=0.4).mean().round()
resampled_time_series_data

,region,total_pickups,avg_pickups
tpep_pickup_datetime,,,
2016-01-01 00:00:00,0,58,58.0
2016-01-01 00:15:00,0,120,97.0
2016-01-01 00:30:00,0,149,123.0
2016-01-01 00:45:00,0,160,140.0
2016-01-01 01:00:00,0,187,161.0
...,...,...,...
2016-03-31 22:45:00,29,14,16.0
2016-03-31 23:00:00,29,17,16.0
2016-03-31 23:15:00,29,18,17.0


In [36]:
resampled_time_series_data.shape

(262080, 3)

In [37]:
# Saving the final data

FINAL_TIME_SERIES_DATA_PATH = "../data/interim/final_time_series_data.csv"
# resampled_time_series_data.to_csv(FINAL_TIME_SERIES_DATA_PATH, index=True)